In [131]:
from fuzzywuzzy import fuzz
import pandas as pd
import json
import pickle

In [132]:
df1 = pd.read_csv("Clubs_data.csv")
df2 = pd.read_csv('Metadata.csv')
df3 = pd.read_csv('Organisers_In_Fests.csv')
df4 = pd.read_csv('Participants_In_Fests.csv')

In [133]:
names2 = df2[df2.columns[0]].values.tolist() # appending all the names in metadata to a list

def join_ratio(s1: str, s2: str) -> int: # This function returns the fuzz ratio between two strings by removing spaces
    return fuzz.ratio(s1.replace(" ", ""), s2.replace(" ", ""))

def set_ratio(s1: str, s2: str) -> int: # This function return the fuzz ratio between two strings without removing spaces
    return fuzz.token_set_ratio(s1, s2)

In [134]:
for i in range(len(df1)): #Replacing all the fuzzy names in clubs dataframe with actual names from metadata frameset
    if len(df1.iloc[i]['Name'])==2 and df1.iloc[i]['Name'].isupper()==True:
        df1.iloc[i]['Name']=df1.iloc[i]['Name'][0]+' '+df1.iloc[i]['Name'][1]        
    s=0
    c=df1.iloc[i]['Name'][0]+df1.iloc[i]['Name'][1:].lower()
    for j in range(len(names2)):
        d=names2[j]
        a=max(set_ratio(c, d), join_ratio(c, d)) #Taking max fuzz ratio by comparing the two string with and without spaces.
            
        if s<a:
            s=a
            b=j
    df1.iloc[i]['Name']=names2[b]

In [135]:
for i in range(len(df3)): #Replacing all the fuzzy names in organisers frame with actual names from metadata dataframe
    if len(df3.iloc[i]['Name'])==2 and df3.iloc[i]['Name'].isupper()==True:
        df3.iloc[i]['Name']=df3.iloc[i]['Name'][0]+' '+df3.iloc[i]['Name'][1] 
    s=0
    c=df3.iloc[i]['Name'][0]+df3.iloc[i]['Name'][1:].lower()
    for j in range(len(names2)):
        d=names2[j]
        a=max(set_ratio(c, d), join_ratio(c, d))
            
        if s<a:
            s=a
            b=j
    df3.iloc[i]['Name']=names2[b]

In [136]:
for i in range(len(df4)): #Replacing all the fuzzy names in participants dataset with actual names from metadata dataframe
    if len(df4.iloc[i]['Name'])==2 and df4.iloc[i]['Name'].isupper()==True:
        df4.iloc[i]['Name']=df4.iloc[i]['Name'][0]+' '+df4.iloc[i]['Name'][1]   
    s=0
    c=df4.iloc[i]['Name'][0]+df4.iloc[i]['Name'][1:].lower()
    for j in range(len(names2)):
        d=names2[j]
        a=max(set_ratio(c, d), join_ratio(c, d))
            
        if s<a:
            s=a
            b=j
    df4.iloc[i]['Name']=names2[b]

In [137]:
df2[['club1_isorganiser','club_1_event_1_participated','club_1_event_2_participated','club_1_event_3_participated','club2_isorganiser','club_2_event_1_participated','club_2_event_2_participated','club_2_event_3_participated','club3_isorganiser','club_3_event_1_participated','club_3_event_2_participated','club_3_event_3_participated']]=pd.DataFrame([['', 'False','False', 'False','', 'False', 'False','False','','False', 'False', 'False']], index=df2.index)

In [138]:
for i in range(len(df1)):  # Merging clubs and metadata dataframes 
    if (df1.iloc[i]['Club_Name'][-1]) == '1':
        a=df1.iloc[i]['Name']
        for j in range(len(df2)):
            if a == df2.iloc[j]['Name']:
                if df1.iloc[i]['Role'] != 'Participant':
                    df2.loc[j]['club1_isorganiser']='organiser'
                 
                elif df2.loc[j]['club1_isorganiser']!='organiser' and df1.iloc[i]['Role'] == 'Participant':
                    df2.loc[j]['club1_isorganiser']=''
                
                b=df1.iloc[i]['Event'][-1]
                df2.loc[j]['club_1_event_'+b+'_participated']='True'
        
        
    elif (df1.iloc[i]['Club_Name'][-1]) == '2':
        a=df1.iloc[i]['Name']
        for j in range(len(df2)):
            if a == df2.iloc[j]['Name']:
                if df1.iloc[i]['Role'] != 'Participant':
                    df2.loc[j]['club2_isorganiser']='organiser'
                 
                elif df2.loc[j]['club2_isorganiser']!='organiser' and df1.iloc[i]['Role'] == 'Participant':
                    df2.loc[j]['club2_isorganiser']=''
                
                b=df1.iloc[i]['Event'][-1]
                df2.loc[j]['club_2_event_'+b+'_participated']='True'
      
    
    elif (df1.iloc[i]['Club_Name'][-1]) == '3':
        a=df1.iloc[i]['Name']
        for j in range(len(df2)):
            if a == df2.iloc[j]['Name']:
                if df1.iloc[i]['Role'] != 'Participant':
                    df2.loc[j]['club3_isorganiser']='organiser'
                 
                elif df2.loc[j]['club3_isorganiser']!='organiser' and df1.iloc[i]['Role'] == 'Participant':
                    df2.loc[j]['club3_isorganiser']=''
                
                b=df1.iloc[i]['Event'][-1]
                df2.loc[j]['club_3_event_'+b+'_participated']='True'


In [139]:
for i in range(1,4): #combining all the clubs columns into one column in metadata dataframe
    for j in range(1,4):
        df2.rename(columns = {'club_'+str(i) +'_event_'+str(j) +'_participated':'Participated'}, inplace = True)
        df2['club_'+str(i) +'_event_'+str(j)]=df2[['Participated']].to_dict('records')
        df2.rename(columns = {'Participated':'club_'+str(i) +'_event_'+str(j) +'_participated'}, inplace = True)
        
    df2.rename(columns = {'club'+str(i)+'_isorganiser':'isOrganiser'}, inplace = True)
    df2['club_'+str(i)]=df2[['isOrganiser','club_'+str(i)+'_event_1','club_'+str(i)+'_event_2','club_'+str(i)+'_event_3']].to_dict('records')
    df2.rename(columns = {'isOrganiser':'club'+str(i)+'_isorganiser'}, inplace = True)
    df2=df2.drop(['club_'+str(i)+'_event_1','club_'+str(i)+'_event_2','club_'+str(i)+'_event_3'],axis=1)
    
    
df2['clubs']=df2[['club_1','club_2','club_3']].to_dict('records')
df2=df2.drop(['club_1','club_2','club_3'],axis=1)

In [140]:
df2[['fest1_isorganiser','fest_1_event_1_participated','fest_1_event_2_participated','fest_1_event_3_participated','fest_1_event_4_participated','fest_1_event_5_participated','fest_1_event_6_participated','fest_1_event_7_participated','fest_1_event_8_participated','fest_1_event_9_participated','fest_1_event_10_participated','fest_1_event_11_participated','fest_1_event_12_participated','fest_1_event_13_participated']]=pd.DataFrame([['','False','False','False','False','False','False','False','False','False','False','False','False','False']],index=df2.index)
df2[['fest2_isorganiser','fest_2_event_1_participated','fest_2_event_2_participated','fest_2_event_3_participated','fest_2_event_4_participated','fest_2_event_5_participated','fest_2_event_6_participated','fest_2_event_7_participated','fest_2_event_8_participated','fest_2_event_9_participated','fest_2_event_10_participated','fest_2_event_11_participated','fest_2_event_12_participated','fest_2_event_13_participated','fest_2_event_14_participated','fest_2_event_15_participated']]=pd.DataFrame([['','False','False','False','False','False','False','False','False','False','False','False','False','False','False','False',]], index=df2.index)


In [141]:
for i in range(len(df3)):# Merging fest organisers and metadata dataframes
    if (df3.iloc[i]['Fest_Name'][-1]) == '1':
        a=df3.iloc[i]['Name']
        for j in range(len(df2)):
            if a == df2.iloc[j]['Name']:
                df2.loc[j]['fest1_isorganiser']='Organiser'
        
    elif (df3.iloc[i]['Fest_Name'][-1]) == '2':
        a=df3.iloc[i]['Name']
        for j in range(len(df2)):
            if a == df2.iloc[j]['Name']:
                df2.loc[j]['fest2_isorganiser']='Organiser'

In [142]:
for i in range(len(df4)): # Merging fest participants and meta dataframe
    if (df4.iloc[i]['Event'][5:6]) == '1':
        a=df4.iloc[i]['Name']
        for j in range(len(df2)):
            if a == df2.iloc[j]['Name']:
                b = df4.iloc[i]['Event'][-1]
                df2.loc[j]['fest_1_event_'+b+'_participated']='True'
                        
    elif (df4.iloc[i]['Event'][5:6]) == '2':
        a=df4.iloc[i]['Name']
        for j in range(len(df2)):
            if a == df2.iloc[j]['Name']:
                b = df4.iloc[i]['Event'][-1]
                df2.loc[j]['fest_2_event_'+b+'_participated']='True'

In [143]:
for j in range(1,14):  #combining all the fests columns into one column in metadata dataframe
    df2.rename(columns = {'fest_1_event_'+str(j) +'_participated':'Participated'}, inplace = True)
    df2['fest_1_event_'+str(j)]=df2[['Participated']].to_dict('records')
    df2.rename(columns = {'Participated':'fest_1_event_'+str(j) +'_participated'}, inplace = True)
    
df2.rename(columns = {'fest1_isorganiser':'isOrganiser'}, inplace = True)
df2['fest_1']=df2[['isOrganiser','fest_1_event_1','fest_1_event_2','fest_1_event_3','fest_1_event_4','fest_1_event_5','fest_1_event_6','fest_1_event_7','fest_1_event_8','fest_1_event_9','fest_1_event_10','fest_1_event_11','fest_1_event_12','fest_1_event_13']].to_dict('records')
df2.rename(columns = {'isOrganiser':'fest1_isorganiser'}, inplace = True)
df2=df2.drop(['fest_1_event_1','fest_1_event_2','fest_1_event_3','fest_1_event_4','fest_1_event_5','fest_1_event_6','fest_1_event_7','fest_1_event_8','fest_1_event_9','fest_1_event_10','fest_1_event_11','fest_1_event_12','fest_1_event_13'],axis=1)

for j in range(1,16):
    df2.rename(columns = {'fest_2_event_'+str(j) +'_participated':'Participated'}, inplace = True)
    df2['fest_2_event_'+str(j)]=df2[['Participated']].to_dict('records')
    df2.rename(columns = {'Participated':'fest_2_event_'+str(j) +'_participated'}, inplace = True)

df2.rename(columns = {'fest2_isorganiser':'isOrganiser'}, inplace = True)
df2['fest_2']=df2[['isOrganiser','fest_2_event_1','fest_2_event_2','fest_2_event_3','fest_2_event_4','fest_2_event_5','fest_2_event_6','fest_2_event_7','fest_2_event_8','fest_2_event_9','fest_2_event_10','fest_2_event_11','fest_2_event_12','fest_2_event_13','fest_2_event_14','fest_2_event_15']].to_dict('records')
df2.rename(columns = {'isOrganiser':'fest2_isorganiser'}, inplace = True)
df2=df2.drop(['fest_2_event_1','fest_2_event_2','fest_2_event_3','fest_2_event_4','fest_2_event_5','fest_2_event_6','fest_2_event_7','fest_2_event_8','fest_2_event_9','fest_2_event_10','fest_2_event_11','fest_2_event_12','fest_2_event_13','fest_2_event_14','fest_2_event_15'],axis=1)
 
df2['fests']=df2[['fest_1','fest_2']].to_dict('records')
df2=df2.drop(['fest_1','fest_2'],axis=1)

In [144]:
out = df2[['Name', 'ID', 'clubs','fests']].to_json(orient='records', indent=4) #converting data from dataframe from to json format 
my_data_file = open('FINALDATA.json', 'w') # creating json format
with open('FINALDATA.json', 'w') as f: 
    f.write(out)

In [145]:
df2['json']=df2[['ID','clubs','fests']].to_dict('records') #creating a dataframe with student names and their profiles
df5=pd.DataFrame(zip(df2['Name'],df2['json']),columns=['Name','json'])

In [146]:
#clubbing columns together
df2['club1'] = df2['club1_isorganiser']+df2['club_1_event_1_participated']+df2['club_1_event_2_participated']+df2['club_1_event_3_participated']
df2['club2'] = df2['club2_isorganiser']+df2['club_2_event_1_participated']+df2['club_2_event_2_participated']+df2['club_2_event_3_participated']
df2['club3'] = df2['club3_isorganiser']+df2['club_3_event_1_participated']+df2['club_3_event_2_participated']+df2['club_3_event_3_participated']
df2['festnum']= df2['fest1_isorganiser']+df2['fest_1_event_1_participated']+df2['fest_1_event_2_participated']+df2[ 'fest_1_event_3_participated']+df2['fest_1_event_4_participated']+ df2['fest_1_event_5_participated']+ df2['fest_1_event_6_participated']+ df2[ 'fest_1_event_7_participated']+ df2['fest_1_event_8_participated']+ df2[ 'fest_1_event_9_participated']+ df2['fest_1_event_10_participated']+ df2[ 'fest_1_event_11_participated']+ df2['fest_1_event_12_participated']+ df2['fest_1_event_13_participated']+ df2['fest2_isorganiser']+ df2[ 'fest_2_event_1_participated']+ df2['fest_2_event_2_participated']+ df2[ 'fest_2_event_3_participated']+ df2['fest_2_event_4_participated']+ df2[ 'fest_2_event_5_participated']+ df2['fest_2_event_6_participated']+ df2['fest_2_event_7_participated']+ df2['fest_2_event_8_participated']+ df2[ 'fest_2_event_9_participated']+ df2['fest_2_event_10_participated']+ df2[ 'fest_2_event_11_participated']+ df2['fest_2_event_12_participated']+ df2['fest_2_event_13_participated']+ df2['fest_2_event_14_participated']+ df2[ 'fest_2_event_15_participated']
df2['clubnum']= df2['club1']+df2['club2']+df2['club3']


In [147]:
#ranking the eligibility of students for general secretary giving respective priority to each factor
gensec = []
for i in range(len(df2)):
    gensec.append((df2.iloc[i]['clubnum'].count('True')/9)*20 + (df2.iloc[i]['festnum'].count('True')/28)*10 + (df2.iloc[i]['clubnum'].count('organiser')/3)*30 + (df2.iloc[i]['festnum'].count('organiser')/2)*40)
general = pd.DataFrame(list(zip(names2,gensec)),columns =['Name', 'Score'])
general=general.sort_values("Score", ascending = False)


In [148]:
#ranking the eligibility of students for club 1 secretary giving respective priority to each factor
club11 = []
for i in range(len(df2)):
    club11.append( (df2.iloc[i]['club1'].count('True')/3)*30 + (df2.iloc[i]['club1'].count('organiser')/1)*70 )
club1sec = pd.DataFrame(list(zip(names2,club11)),columns =['Name', 'Score'])
club1sec=club1sec.sort_values("Score", ascending = False)


In [149]:
#ranking the eligibility of students for club 2 secretary giving respective priority to each factor
club22 = []
for i in range(len(df2)):
    club22.append((df2.iloc[i]['club2'].count('True')/3)*30 + (df2.iloc[i]['club2'].count('organiser')/1)*70 )
club2sec = pd.DataFrame(list(zip(names2,club22)),columns =['Name', 'Score'])
club2sec=club2sec.sort_values("Score", ascending = False)


In [150]:
#ranking the eligibility of students for club 3 secretary giving respective priority to each factor
club33 = []
for i in range(len(df2)):
    club33.append( (df2.iloc[i]['club3'].count('True')/3)*30 + (df2.iloc[i]['club3'].count('organiser')/1)*70 )
club3sec = pd.DataFrame(list(zip(names2,club33)),columns =['Name', 'Score'])
club3sec=club3sec.sort_values("Score", ascending = False)


In [151]:
#ranking the eligibility of students for clutural secretary giving respective priority to each factor
festsum = []
for i in range(len(df2)):
    festsum.append( (df2.iloc[i]['festnum'].count('True')/28)*10 + (df2.iloc[i]['festnum'].count('Organiser')/2)*90 )
cultsec = pd.DataFrame(list(zip(names2,festsum)),columns =['Name', 'Score'])
cultsec=cultsec.sort_values("Score", ascending = False)


In [152]:
#converting dataframes to pkl files
pickle.dump(general,open('general.pkl','wb'))
pickle.dump(club1sec,open('club1sec.pkl','wb'))
pickle.dump(club2sec,open('club2sec.pkl','wb'))
pickle.dump(club3sec,open('club3sec.pkl','wb'))
pickle.dump(cultsec,open('cultsec.pkl','wb'))
pickle.dump(df5,open('profile.pkl','wb'))